In [1]:
class Patient(object):
    # __init__ is a special method called whenever you try to make
    # an instance of a class. As you heard, it initializes the object.
    # Here, we'll initialize some of the data.
    def __init__(self):
        # Let's add some data to the [instance of the] class.
        self.age = 60
        self.timeontreatment = 0
        self.alive = True

    # We can also add our own functions. When our ball bounces,
    # its vertical velocity will be negated. (no gravity here!)
    def treat(self):
        self.timeontreatment += 1
        if self.timeontreatment > 28:
            self.alive = False


p0 = Patient()

# game loop
for i in range(5):
    p0.treat()

p0.timeontreatment, p0.alive

(5, True)

https://python-course.eu/applications-python/finite-state-machine.php

In [2]:
class StateMachine:
    
    def __init__(self):
        self.handlers = {}
        self.startState = None
        self.endStates = []

    def add_state(self, name, handler, end_state=0):
        name = name.upper()
        self.handlers[name] = handler
        if end_state:
            self.endStates.append(name)

    def set_start(self, name):
        self.startState = name.upper()

    def run(self, cargo):
        try:
            handler = self.handlers[self.startState]
        except:
            raise InitializationError("must call .set_start() before .run()")
        if not self.endStates:
            raise  InitializationError("at least one state must be an end_state")
    
        while True:
            (newState, cargo) = handler(cargo)
            if newState.upper() in self.endStates:
                print("reached ", newState)
                break 
            else:
                handler = self.handlers[newState.upper()] 

In [3]:
positive_adjectives = ["great","super", "fun", "entertaining", "easy"]
negative_adjectives = ["boring", "difficult", "ugly", "bad"]

def start_transitions(txt):
    splitted_txt = txt.split(None,1)
    word, txt = splitted_txt if len(splitted_txt) > 1 else (txt,"")
    if word == "Python":
        newState = "Python_state"
    else:
        newState = "error_state"
    return (newState, txt)

def python_state_transitions(txt):
    splitted_txt = txt.split(None,1)
    word, txt = splitted_txt if len(splitted_txt) > 1 else (txt,"")
    if word == "is":
        newState = "is_state"
    else:
        newState = "error_state"
    return (newState, txt)

def is_state_transitions(txt):
    splitted_txt = txt.split(None,1)
    word, txt = splitted_txt if len(splitted_txt) > 1 else (txt,"")
    if word == "not":
        newState = "not_state"
    elif word in positive_adjectives:
        newState = "pos_state"
    elif word in negative_adjectives:
        newState = "neg_state"
    else:
        newState = "error_state"
    return (newState, txt)

def not_state_transitions(txt):
    splitted_txt = txt.split(None,1)
    word, txt = splitted_txt if len(splitted_txt) > 1 else (txt,"")
    if word in positive_adjectives:
        newState = "neg_state"
    elif word in negative_adjectives:
        newState = "pos_state"
    else:
        newState = "error_state"
    return (newState, txt)

def neg_state(txt):
    print("Hallo")
    return ("neg_state", "")


m = StateMachine()
m.add_state("Start", start_transitions)
m.add_state("Python_state", python_state_transitions)
m.add_state("is_state", is_state_transitions)
m.add_state("not_state", not_state_transitions)
m.add_state("neg_state", None, end_state=1)
m.add_state("pos_state", None, end_state=1)
m.add_state("error_state", None, end_state=1)
m.set_start("Start")
m.run("Python is great")
m.run("Python is difficult")
m.run("Perl is ugly")

reached  pos_state
reached  neg_state
reached  error_state


I've created a clean, extensible Python framework for modeling patient treatment flows as finite state machines. Here are the key features:

**Core Components:**
1. **Patient class** - Each patient is an FSM tracking their current state, time spent in each state, and complete history
2. **TransitionRule class** - Defines when/how patients move between states with configurable probabilities and conditions
3. **TreatmentFlowModel class** - Orchestrates the simulation and manages all patients

**Key Design Decisions:**
- **Flexible transitions**: Rules support minimum durations, probabilities, and custom conditions
- **Complete history tracking**: Every patient's journey is recorded for analysis
- **Easy data readout**: Built-in methods for state distributions, outcomes, journey details, and statistics
- **Extensibility**: Adding new states or transition rules is straightforward

**To Adapt This Framework:**
1. **Add states**: Extend the `PatientState` enum
2. **Add transition rules**: Create new `TransitionRule` objects with your logic
3. **Custom conditions**: Use the `condition` parameter for complex transition logic
4. **Add data readouts**: The model class makes it easy to add new analysis methods

The example simulates 100 patients through a 3-treatment pathway with various outcomes (remission or death). Run it to see summary statistics and individual patient journeys!

In [4]:
"""
Patient Treatment Flow Framework using Finite State Machines

This framework models patient journeys through treatment as FSMs, allowing 
easy tracking of states, transitions, and outcomes over time.
"""

from dataclasses import dataclass, field
from typing import Dict, List, Optional, Callable
from enum import Enum
import random


# ============================================================================
# STATE DEFINITIONS
# ============================================================================

class PatientState(Enum):
    """Define all possible patient states"""
    DIAGNOSED = "diagnosed"
    TREATMENT_1 = "treatment_1"
    TREATMENT_2 = "treatment_2"
    TREATMENT_3 = "treatment_3"
    REMISSION = "remission"
    DECEASED = "deceased"


# ============================================================================
# PATIENT CLASS (Finite State Machine)
# ============================================================================

@dataclass
class Patient:
    """
    Represents a patient as a finite state machine.
    Tracks current state, history, and time spent in each state.
    """
    id: int
    current_state: PatientState = PatientState.DIAGNOSED
    time_in_current_state: int = 0
    state_history: List[tuple] = field(default_factory=list)  # (state, duration)
    total_time: int = 0
    
    def transition_to(self, new_state: PatientState):
        """
        Move patient to a new state, recording history.
        """
        # Record the state we're leaving
        self.state_history.append((self.current_state, self.time_in_current_state))
        
        # Transition to new state
        self.current_state = new_state
        self.time_in_current_state = 0
    
    def advance_time(self, time_units: int = 1):
        """
        Progress time for this patient in their current state.
        """
        self.time_in_current_state += time_units
        self.total_time += time_units
    
    def is_terminal(self) -> bool:
        """
        Check if patient is in a terminal state (deceased or long-term remission).
        """
        return self.current_state in [PatientState.DECEASED, PatientState.REMISSION]


# ============================================================================
# TRANSITION RULES
# ============================================================================

@dataclass
class TransitionRule:
    """
    Defines when and how a patient moves from one state to another.
    """
    from_state: PatientState
    to_state: PatientState
    min_duration: int  # Minimum time before transition can occur
    probability: float  # Probability of transition per time unit after min_duration
    condition: Optional[Callable[[Patient], bool]] = None  # Optional custom condition
    
    def can_transition(self, patient: Patient) -> bool:
        """
        Check if this transition rule applies to the patient.
        """
        # Must be in the correct state
        if patient.current_state != self.from_state:
            return False
        
        # Must have spent minimum time in state
        if patient.time_in_current_state < self.min_duration:
            return False
        
        # Check custom condition if provided
        if self.condition and not self.condition(patient):
            return False
        
        # Apply probability
        return random.random() < self.probability
    

# ============================================================================
# FLOW MODEL
# ============================================================================

class TreatmentFlowModel:
    """
    Orchestrates patient treatment flows by defining transition rules
    and simulating patient journeys through states.
    """
    
    def __init__(self):
        self.patients: List[Patient] = []
        self.transition_rules: List[TransitionRule] = []
        self.current_time: int = 0
        
    def add_transition_rule(self, rule: TransitionRule):
        """Add a transition rule to the model"""
        self.transition_rules.append(rule)
    
    def add_patient(self, patient: Patient):
        """Add a patient to the simulation"""
        self.patients.append(patient)
    
    def step(self):
        """
        Advance simulation by one time unit.
        For each patient, check transitions and advance time.
        """
        self.current_time += 1
        
        for patient in self.patients:
            # Skip patients in terminal states
            if patient.is_terminal():
                continue
            
            # Check all transition rules to see if patient should change state
            transitioned = False
            for rule in self.transition_rules:
                if rule.can_transition(patient):
                    patient.transition_to(rule.to_state)
                    transitioned = True
                    break  # Only one transition per time step
            
            # Always advance time for the patient
            patient.advance_time(1)
    
    def run(self, max_time: int):
        """
        Run the simulation for a specified number of time units.
        """
        for _ in range(max_time):
            self.step()
    
    # ========================================================================
    # DATA READOUT METHODS
    # ========================================================================
    
    def get_state_distribution(self) -> Dict[PatientState, int]:
        """Get current count of patients in each state"""
        distribution = {state: 0 for state in PatientState}
        for patient in self.patients:
            distribution[patient.current_state] += 1
        return distribution
    
    def get_average_time_in_state(self, state: PatientState) -> float:
        """Calculate average time patients spend in a given state"""
        times = []
        for patient in self.patients:
            for hist_state, duration in patient.state_history:
                if hist_state == state:
                    times.append(duration)
        return sum(times) / len(times) if times else 0
    
    def get_patient_outcomes(self) -> Dict[str, int]:
        """Summarize final outcomes for all patients"""
        outcomes = {"deceased": 0, "remission": 0, "in_treatment": 0}
        for patient in self.patients:
            if patient.current_state == PatientState.DECEASED:
                outcomes["deceased"] += 1
            elif patient.current_state == PatientState.REMISSION:
                outcomes["remission"] += 1
            else:
                outcomes["in_treatment"] += 1
        return outcomes
    
    def get_patient_journeys(self) -> List[Dict]:
        """Get detailed journey information for each patient"""
        journeys = []
        for patient in self.patients:
            journey = {
                "patient_id": patient.id,
                "current_state": patient.current_state.value,
                "total_time": patient.total_time,
                "history": [(s.value, d) for s, d in patient.state_history]
            }
            journeys.append(journey)
        return journeys
    
    def print_summary(self):
        """Print a summary of the simulation results"""
        print(f"\n{'='*60}")
        print(f"SIMULATION SUMMARY (Time: {self.current_time})")
        print(f"{'='*60}")
        
        print(f"\nTotal Patients: {len(self.patients)}")
        
        print("\nCurrent State Distribution:")
        for state, count in self.get_state_distribution().items():
            print(f"  {state.value:20s}: {count:4d} patients")
        
        print("\nPatient Outcomes:")
        for outcome, count in self.get_patient_outcomes().items():
            print(f"  {outcome:20s}: {count:4d} patients")
        
        print("\nAverage Time in Each State:")
        for state in PatientState:
            avg_time = self.get_average_time_in_state(state)
            print(f"  {state.value:20s}: {avg_time:6.1f} time units")


# ============================================================================
# EXAMPLE USAGE
# ============================================================================

def create_example_model():
    """
    Create an example treatment flow model with realistic transitions.
    This demonstrates how to set up a simple treatment pathway.
    """
    model = TreatmentFlowModel()
    
    # Define transition rules for a typical treatment flow
    # Format: from_state -> to_state (min_duration, probability per time unit)
    
    # From diagnosis to first treatment (immediate, certain)
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.DIAGNOSED,
        to_state=PatientState.TREATMENT_1,
        min_duration=0,
        probability=1.0
    ))
    
    # Treatment 1 outcomes (after minimum 3 time units)
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_1,
        to_state=PatientState.TREATMENT_2,
        min_duration=3,
        probability=0.15  # 15% chance per time unit to progress
    ))
    
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_1,
        to_state=PatientState.DECEASED,
        min_duration=3,
        probability=0.02  # 2% mortality risk per time unit
    ))
    
    # Treatment 2 outcomes
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_2,
        to_state=PatientState.TREATMENT_3,
        min_duration=4,
        probability=0.10
    ))
    
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_2,
        to_state=PatientState.REMISSION,
        min_duration=4,
        probability=0.08  # 8% chance of remission per time unit
    ))
    
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_2,
        to_state=PatientState.DECEASED,
        min_duration=4,
        probability=0.03
    ))
    
    # Treatment 3 outcomes
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_3,
        to_state=PatientState.REMISSION,
        min_duration=5,
        probability=0.12
    ))
    
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_3,
        to_state=PatientState.DECEASED,
        min_duration=5,
        probability=0.05
    ))
    
    return model


if __name__ == "__main__":
    # Create the model
    model = create_example_model()
    
    # Add 100 patients to the simulation
    for i in range(100):
        model.add_patient(Patient(id=i))
    
    # Run simulation for 50 time units
    print("Running simulation...")
    model.run(max_time=50)
    
    # Display results
    model.print_summary()
    
    # Example: Get detailed journey for first patient
    print("\n" + "="*60)
    print("EXAMPLE: First Patient's Journey")
    print("="*60)
    journeys = model.get_patient_journeys()
    first_journey = journeys[0]
    print(f"Patient {first_journey['patient_id']}:")
    print(f"  Current State: {first_journey['current_state']}")
    print(f"  Total Time: {first_journey['total_time']}")
    print(f"  Journey:")
    for state, duration in first_journey['history']:
        print(f"    {state:20s} for {duration} time units")

Running simulation...

SIMULATION SUMMARY (Time: 50)

Total Patients: 100

Current State Distribution:
  diagnosed           :    0 patients
  treatment_1         :    0 patients
  treatment_2         :    0 patients
  treatment_3         :    2 patients
  remission           :   60 patients
  deceased            :   38 patients

Patient Outcomes:
  deceased            :   38 patients
  remission           :   60 patients
  in_treatment        :    2 patients

Average Time in Each State:
  diagnosed           :    0.0 time units
  treatment_1         :    8.0 time units
  treatment_2         :    8.3 time units
  treatment_3         :   10.5 time units
  remission           :    0.0 time units
  deceased            :    0.0 time units

EXAMPLE: First Patient's Journey
Patient 0:
  Current State: deceased
  Total Time: 23
  Journey:
    diagnosed            for 0 time units
    treatment_1          for 18 time units
    treatment_2          for 4 time units


### Patient criteria

- SLIM-CRAB criterion
- fit for HD
- past treatments

### Patient states

- line of treatment
- treatment phase
- current treatment
- recurring




In [8]:
"""
Patient Treatment Flow Framework using Finite State Machines

This framework models patient journeys through treatment as FSMs, allowing 
easy tracking of states, transitions, and outcomes over time.
"""

from dataclasses import dataclass, field
from typing import Dict, List, Optional, Callable
from enum import Enum
import random


# ============================================================================
# STATE DEFINITIONS
# ============================================================================

class PatientState(Enum):
    """Define all possible patient states"""
    LINE = "diagnosed"
    TREATMENT_1 = "treatment_1"
    TREATMENT_2 = "treatment_2"
    TREATMENT_3 = "treatment_3"
    REMISSION = "remission"
    DECEASED = "deceased"


# ============================================================================
# PATIENT CLASS (Finite State Machine)
# ============================================================================

@dataclass
class Patient:
    """
    Represents a patient as a finite state machine.
    Tracks current state, history, and time spent in each state.
    """
    id: int
    current_state: PatientState = PatientState.LINE
    time_in_current_state: int = 0
    state_history: List[tuple] = field(default_factory=list)  # (state, duration)
    total_time: int = 0
    
    def transition_to(self, new_state: PatientState):
        """
        Move patient to a new state, recording history.
        """
        # Record the state we're leaving
        self.state_history.append((self.current_state, self.time_in_current_state))
        
        # Transition to new state
        self.current_state = new_state
        self.time_in_current_state = 0
    
    def advance_time(self, time_units: int = 1):
        """
        Progress time for this patient in their current state.
        """
        self.time_in_current_state += time_units
        self.total_time += time_units
    
    def is_terminal(self) -> bool:
        """
        Check if patient is in a terminal state (deceased or long-term remission).
        """
        return self.current_state in [PatientState.DECEASED, PatientState.REMISSION]


# ============================================================================
# TRANSITION RULES
# ============================================================================

@dataclass
class TransitionRule:
    """
    Defines when and how a patient moves from one state to another.
    """
    from_state: PatientState
    to_state: PatientState
    min_duration: int  # Minimum time before transition can occur
    probability: float  # Probability of transition per time unit after min_duration
    condition: Optional[Callable[[Patient], bool]] = None  # Optional custom condition
    
    def can_transition(self, patient: Patient) -> bool:
        """
        Check if this transition rule applies to the patient.
        """
        # Must be in the correct state
        if patient.current_state != self.from_state:
            return False
        
        # Must have spent minimum time in state
        if patient.time_in_current_state < self.min_duration:
            return False
        
        # Check custom condition if provided
        if self.condition and not self.condition(patient):
            return False
        
        # Apply probability
        return random.random() < self.probability
    

# ============================================================================
# FLOW MODEL
# ============================================================================

class TreatmentFlowModel:
    """
    Orchestrates patient treatment flows by defining transition rules
    and simulating patient journeys through states.
    """
    
    def __init__(self):
        self.patients: List[Patient] = []
        self.transition_rules: List[TransitionRule] = []
        self.current_time: int = 0
        
    def add_transition_rule(self, rule: TransitionRule):
        """Add a transition rule to the model"""
        self.transition_rules.append(rule)
    
    def add_patient(self, patient: Patient):
        """Add a patient to the simulation"""
        self.patients.append(patient)
    
    def step(self):
        """
        Advance simulation by one time unit.
        For each patient, check transitions and advance time.
        """
        self.current_time += 1
        
        for patient in self.patients:
            # Skip patients in terminal states
            if patient.is_terminal():
                continue
            
            # Check all transition rules to see if patient should change state
            transitioned = False
            for rule in self.transition_rules:
                if rule.can_transition(patient):
                    patient.transition_to(rule.to_state)
                    transitioned = True
                    break  # Only one transition per time step
            
            # Always advance time for the patient
            patient.advance_time(1)
    
    def run(self, max_time: int):
        """
        Run the simulation for a specified number of time units.
        """
        for _ in range(max_time):
            self.step()
    
    # ========================================================================
    # DATA READOUT METHODS
    # ========================================================================
    
    def get_state_distribution(self) -> Dict[PatientState, int]:
        """Get current count of patients in each state"""
        distribution = {state: 0 for state in PatientState}
        for patient in self.patients:
            distribution[patient.current_state] += 1
        return distribution
    
    def get_average_time_in_state(self, state: PatientState) -> float:
        """Calculate average time patients spend in a given state"""
        times = []
        for patient in self.patients:
            for hist_state, duration in patient.state_history:
                if hist_state == state:
                    times.append(duration)
        return sum(times) / len(times) if times else 0
    
    def get_patient_outcomes(self) -> Dict[str, int]:
        """Summarize final outcomes for all patients"""
        outcomes = {"deceased": 0, "remission": 0, "in_treatment": 0}
        for patient in self.patients:
            if patient.current_state == PatientState.DECEASED:
                outcomes["deceased"] += 1
            elif patient.current_state == PatientState.REMISSION:
                outcomes["remission"] += 1
            else:
                outcomes["in_treatment"] += 1
        return outcomes
    
    def get_patient_journeys(self) -> List[Dict]:
        """Get detailed journey information for each patient"""
        journeys = []
        for patient in self.patients:
            journey = {
                "patient_id": patient.id,
                "current_state": patient.current_state.value,
                "total_time": patient.total_time,
                "history": [(s.value, d) for s, d in patient.state_history]
            }
            journeys.append(journey)
        return journeys
    
    def print_summary(self):
        """Print a summary of the simulation results"""
        print(f"\n{'='*60}")
        print(f"SIMULATION SUMMARY (Time: {self.current_time})")
        print(f"{'='*60}")
        
        print(f"\nTotal Patients: {len(self.patients)}")
        
        print("\nCurrent State Distribution:")
        for state, count in self.get_state_distribution().items():
            print(f"  {state.value:20s}: {count:4d} patients")
        
        print("\nPatient Outcomes:")
        for outcome, count in self.get_patient_outcomes().items():
            print(f"  {outcome:20s}: {count:4d} patients")
        
        print("\nAverage Time in Each State:")
        for state in PatientState:
            avg_time = self.get_average_time_in_state(state)
            print(f"  {state.value:20s}: {avg_time:6.1f} time units")


# ============================================================================
# EXAMPLE USAGE
# ============================================================================

def create_example_model():
    """
    Create an example treatment flow model with realistic transitions.
    This demonstrates how to set up a simple treatment pathway.
    """
    model = TreatmentFlowModel()
    
    # Define transition rules for a typical treatment flow
    # Format: from_state -> to_state (min_duration, probability per time unit)

    # Simple usage matching your original structure

    generator = RandomTransitionRuleGenerator()
    
    model.add_transition_rule(generator.generate_rule(
        from_state=PatientState.LINE,
        to_state=PatientState.TREATMENT_1,
        prob_distribution=('uniform', 0.8, 1.0),  # 80-100% probability
        duration_distribution=('triangular', 0, 7, 14)  # 0-14 days, most likely 7
    ))
    
    # From diagnosis to first treatment (immediate, certain)
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.LINE,
        to_state=PatientState.TREATMENT_1,
        min_duration=0,
        probability=1.0
    ))
    
    # Treatment 1 outcomes (after minimum 3 time units)
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_1,
        to_state=PatientState.TREATMENT_2,
        min_duration=3,
        probability=0.15  # 15% chance per time unit to progress
    ))
    
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_1,
        to_state=PatientState.DECEASED,
        min_duration=3,
        probability=0.02  # 2% mortality risk per time unit
    ))
    
    # Treatment 2 outcomes
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_2,
        to_state=PatientState.TREATMENT_3,
        min_duration=4,
        probability=0.10
    ))
    
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_2,
        to_state=PatientState.REMISSION,
        min_duration=4,
        probability=0.08  # 8% chance of remission per time unit
    ))
    
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_2,
        to_state=PatientState.DECEASED,
        min_duration=4,
        probability=0.03
    ))
    
    # Treatment 3 outcomes
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_3,
        to_state=PatientState.REMISSION,
        min_duration=5,
        probability=0.12
    ))
    
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_3,
        to_state=PatientState.DECEASED,
        min_duration=5,
        probability=0.05
    ))
    
    return model


if __name__ == "__main__":
    # Create the model
    model = create_example_model()
    
    # Add 100 patients to the simulation
    for i in range(100):
        model.add_patient(Patient(id=i))
    
    # Run simulation for 50 time units
    print("Running simulation...")
    model.run(max_time=50)
    
    # Display results
    model.print_summary()
    
    # Example: Get detailed journey for first patient
    print("\n" + "="*60)
    print("EXAMPLE: First Patient's Journey")
    print("="*60)
    journeys = model.get_patient_journeys()
    first_journey = journeys[0]
    print(f"Patient {first_journey['patient_id']}:")
    print(f"  Current State: {first_journey['current_state']}")
    print(f"  Total Time: {first_journey['total_time']}")
    print(f"  Journey:")
    for state, duration in first_journey['history']:
        print(f"    {state:20s} for {duration} time units")

Running simulation...

SIMULATION SUMMARY (Time: 50)

Total Patients: 100

Current State Distribution:
  diagnosed           :    0 patients
  treatment_1         :    0 patients
  treatment_2         :    1 patients
  treatment_3         :    0 patients
  remission           :   62 patients
  deceased            :   37 patients

Patient Outcomes:
  deceased            :   37 patients
  remission           :   62 patients
  in_treatment        :    1 patients

Average Time in Each State:
  diagnosed           :    0.0 time units
  treatment_1         :    8.3 time units
  treatment_2         :    7.9 time units
  treatment_3         :    9.0 time units
  remission           :    0.0 time units
  deceased            :    0.0 time units

EXAMPLE: First Patient's Journey
Patient 0:
  Current State: remission
  Total Time: 8
  Journey:
    diagnosed            for 0 time units
    treatment_1          for 3 time units
    treatment_2          for 4 time units


In [10]:
import random
from enum import Enum
from typing import Optional, Union, Callable
import numpy as np


class PatientState(Enum):
    LINE = "line"
    TREATMENT_1 = "treatment_1"
    TREATMENT_2 = "treatment_2"
    PROGRESSION = "progression"
    DEATH = "death"

class TransitionRule:
    def __init__(self, from_state, to_state, min_duration, probability):
        self.from_state = from_state
        self.to_state = to_state
        self.min_duration = min_duration
        self.probability = probability
    
    def __repr__(self):
        return (f"TransitionRule(from_state={self.from_state}, to_state={self.to_state}, "
                f"min_duration={self.min_duration}, probability={self.probability})")

class RandomTransitionRuleGenerator:
    """
    Generates TransitionRule objects with parameters sampled from specified distributions.
    
    Example usage:
        generator = RandomTransitionRuleGenerator()
        
        # Create a rule with uniform probability and triangular duration
        rule = generator.generate_rule(
            from_state=PatientState.LINE,
            to_state=PatientState.TREATMENT_1,
            prob_distribution=('uniform', 0.8, 1.0),
            duration_distribution=('triangular', 0, 30, 15)
        )
    """
    
    @staticmethod
    def sample_from_distribution(distribution_spec):
        """
        Sample a value from a specified distribution.
        
        Args:
            distribution_spec: Tuple of (distribution_name, *parameters)
                Supported distributions:
                - ('uniform', min, max)
                - ('normal', mean, std)
                - ('triangular', left, mode, right)
                - ('beta', alpha, beta)
                - ('exponential', scale)
                - ('fixed', value)
                - ('bernoulli', p)  # returns 0 or 1
        
        Returns:
            Sampled value
        """
        if not distribution_spec:
            raise ValueError("Distribution specification required")
        
        dist_name = distribution_spec[0].lower()
        params = distribution_spec[1:]
        
        if dist_name == 'uniform':
            min_val, max_val = params
            return random.uniform(min_val, max_val)
        
        elif dist_name == 'normal':
            mean, std = params
            # Clip to avoid extreme values
            value = np.random.normal(mean, std)
            # Ensure non-negative for probabilities/durations
            return max(0, value)
        
        elif dist_name == 'triangular':
            left, mode, right = params
            return random.triangular(left, mode, right)
        
        elif dist_name == 'beta':
            alpha, beta = params
            return np.random.beta(alpha, beta)
        
        elif dist_name == 'exponential':
            scale, = params
            return np.random.exponential(scale)
        
        elif dist_name == 'fixed':
            value, = params
            return value
        
        elif dist_name == 'bernoulli':
            p, = params
            return 1 if random.random() < p else 0
        
        elif dist_name == 'discrete':
            # params: list of (value, weight) pairs
            values, weights = zip(*params)
            return random.choices(values, weights=weights, k=1)[0]
        
        elif dist_name == 'poisson':
            lam, = params
            return np.random.poisson(lam)
        
        else:
            raise ValueError(f"Unsupported distribution: {dist_name}")
    
    @staticmethod
    def generate_rule(
        from_state: PatientState,
        to_state: PatientState,
        prob_distribution: Optional[tuple] = None,
        duration_distribution: Optional[tuple] = None,
        probability_clip_range: tuple = (0.0, 1.0),
        duration_clip_range: tuple = (0, None)
    ) -> TransitionRule:
        """
        Generate a TransitionRule with randomly sampled parameters.
        
        Args:
            from_state: Starting state
            to_state: Target state
            prob_distribution: Distribution specification for probability
            duration_distribution: Distribution specification for min_duration
            probability_clip_range: (min, max) to clip probability values
            duration_clip_range: (min, max) to clip duration values
        
        Returns:
            TransitionRule with sampled parameters
        """
        # Sample probability
        if prob_distribution:
            prob = RandomTransitionRuleGenerator.sample_from_distribution(prob_distribution)
            # Clip to valid probability range
            prob_min, prob_max = probability_clip_range
            prob = max(prob_min, min(prob_max, prob))
            # Round for readability
            prob = round(prob, 3)
        else:
            prob = 1.0  # Default if no distribution specified
        
        # Sample duration
        if duration_distribution:
            duration = RandomTransitionRuleGenerator.sample_from_distribution(duration_distribution)
            # Clip to valid duration range
            dur_min, dur_max = duration_clip_range
            if dur_min is not None:
                duration = max(dur_min, duration)
            if dur_max is not None:
                duration = min(dur_max, duration)
            # Round to integer for durations
            duration = int(round(duration))
        else:
            duration = 0  # Default if no distribution specified
        
        return TransitionRule(
            from_state=from_state,
            to_state=to_state,
            min_duration=duration,
            probability=prob
        )


# ============================================================================
# EXAMPLE USAGE
# ============================================================================

# Example 1: Your original use case with random parameters
def example_original_use_case():
    """Example matching your original code structure with random parameters."""
    # Create generator
    generator = RandomTransitionRuleGenerator()
    
    # Original transition with deterministic parameters
    original_rule = TransitionRule(
        from_state=PatientState.LINE,
        to_state=PatientState.TREATMENT_1,
        min_duration=0,
        probability=1.0
    )
    
    print("Original rule:")
    print(f"  {original_rule}")
    
    # Random version 1: Uniform probability, fixed duration
    random_rule_1 = generator.generate_rule(
        from_state=PatientState.LINE,
        to_state=PatientState.TREATMENT_1,
        prob_distribution=('uniform', 0.7, 0.95),  # 70-95% probability
        duration_distribution=('fixed', 0)  # Always 0 duration
    )
    
    print("\nRandom rule 1 (uniform probability):")
    print(f"  {random_rule_1}")
    
    # Random version 2: Beta distribution for probability, triangular for duration
    random_rule_2 = generator.generate_rule(
        from_state=PatientState.LINE,
        to_state=PatientState.TREATMENT_1,
        prob_distribution=('beta', 2, 5),  # Beta(2,5) - skewed toward lower probabilities
        duration_distribution=('triangular', 0, 7, 14)  # 0-14 days, mode at 7
    )
    
    print("\nRandom rule 2 (beta probability, triangular duration):")
    print(f"  {random_rule_2}")
    
    # Random version 3: Normal distribution for both
    random_rule_3 = generator.generate_rule(
        from_state=PatientState.LINE,
        to_state=PatientState.TREATMENT_1,
        prob_distribution=('normal', 0.85, 0.1),  # Mean 0.85, std 0.1
        duration_distribution=('normal', 5, 3)  # Mean 5, std 3 days
    )
    
    print("\nRandom rule 3 (normal distributions):")
    print(f"  {random_rule_3}")
    
    return [original_rule, random_rule_1, random_rule_2, random_rule_3]


# Example 2: Multiple transition rules for a full model
def example_full_model_transitions():
    """Generate multiple random transition rules for a complete patient pathway."""
    generator = RandomTransitionRuleGenerator()
    
    # Define transition distributions for different state transitions
    transition_specs = [
        {
            'from_state': PatientState.LINE,
            'to_state': PatientState.TREATMENT_1,
            'prob_distribution': ('uniform', 0.9, 1.0),
            'duration_distribution': ('triangular', 0, 14, 28)
        },
        {
            'from_state': PatientState.TREATMENT_1,
            'to_state': PatientState.TREATMENT_2,
            'prob_distribution': ('beta', 3, 7),  # 30% chance of moving to second line
            'duration_distribution': ('normal', 180, 45)  # ~6 months with variation
        },
        {
            'from_state': PatientState.TREATMENT_1,
            'to_state': PatientState.PROGRESSION,
            'prob_distribution': ('beta', 5, 5),  # 50% chance of progression
            'duration_distribution': ('exponential', 365)  # Mean time to progression 1 year
        },
        {
            'from_state': PatientState.TREATMENT_2,
            'to_state': PatientState.PROGRESSION,
            'prob_distribution': ('uniform', 0.7, 0.9),
            'duration_distribution': ('normal', 120, 30)  # ~4 months
        },
        {
            'from_state': PatientState.PROGRESSION,
            'to_state': PatientState.DEATH,
            'prob_distribution': ('uniform', 0.8, 1.0),
            'duration_distribution': ('triangular', 30, 90, 180)  # 1-6 months
        }
    ]
    
    rules = []
    for spec in transition_specs:
        rule = generator.generate_rule(**spec)
        rules.append(rule)
        print(f"{spec['from_state'].value} → {spec['to_state'].value}: "
              f"prob={rule.probability:.3f}, duration={rule.min_duration}")
    
    return rules


# Example 3: Batch generation for sensitivity analysis
def example_batch_generation(num_samples: int = 5):
    """Generate multiple samples of the same transition for sensitivity analysis."""
    generator = RandomTransitionRuleGenerator()
    
    print(f"Generating {num_samples} random samples for LINE → TREATMENT_1:")
    print("-" * 60)
    
    rules = []
    for i in range(num_samples):
        rule = generator.generate_rule(
            from_state=PatientState.LINE,
            to_state=PatientState.TREATMENT_1,
            prob_distribution=('normal', 0.85, 0.05),
            duration_distribution=('triangular', 0, 7, 21)
        )
        rules.append(rule)
        print(f"Sample {i+1}: prob={rule.probability:.3f}, duration={rule.min_duration}")
    
    # Calculate statistics
    probs = [r.probability for r in rules]
    durations = [r.min_duration for r in rules]
    
    print("\nStatistics:")
    print(f"  Probability: mean={np.mean(probs):.3f}, std={np.std(probs):.3f}, "
          f"range=[{min(probs):.3f}, {max(probs):.3f}]")
    print(f"  Duration: mean={np.mean(durations):.1f}, std={np.std(durations):.1f}, "
          f"range=[{min(durations)}, {max(durations)}]")
    
    return rules


# ============================================================================
# WRAPPER FUNCTION FOR YOUR SPECIFIC USE CASE
# ============================================================================

def create_random_transition_rule(
    from_state: PatientState,
    to_state: PatientState,
    model,  # Your model object
    probability_config: Optional[dict] = None,
    duration_config: Optional[dict] = None
):
    """
    Wrapper function that creates and adds a random transition rule to your model.
    
    Args:
        from_state: Starting state
        to_state: Target state
        model: Your model object (must have add_transition_rule method)
        probability_config: Configuration for probability distribution
        duration_config: Configuration for duration distribution
    """
    # Default configurations if none provided
    if probability_config is None:
        probability_config = {
            'distribution': ('uniform', 0.7, 1.0),  # 70-100% probability
            'clip_range': (0.0, 1.0)
        }
    
    if duration_config is None:
        duration_config = {
            'distribution': ('triangular', 0, 0, 14),  # 0-14 days, mode at 0
            'clip_range': (0, None)
        }
    
    # Generate random rule
    generator = RandomTransitionRuleGenerator()
    rule = generator.generate_rule(
        from_state=from_state,
        to_state=to_state,
        prob_distribution=probability_config['distribution'],
        duration_distribution=duration_config['distribution'],
        probability_clip_range=probability_config['clip_range'],
        duration_clip_range=duration_config['clip_range']
    )
    
    # Add to model
    model.add_transition_rule(rule)
    
    print(f"Added random transition rule: {rule}")
    return rule


# ============================================================================
# MAIN EXECUTION (for testing)
# ============================================================================

if __name__ == "__main__":
    print("=== EXAMPLE 1: Original use case with random parameters ===")
    rules1 = example_original_use_case()
    
    print("\n\n=== EXAMPLE 2: Full model transitions ===")
    rules2 = example_full_model_transitions()
    
    print("\n\n=== EXAMPLE 3: Batch generation for sensitivity ===")
    rules3 = example_batch_generation(10)
    
    print("\n\n=== Quick usage example ===")
    # Simulate a model class for demonstration
    class MockModel:
        def __init__(self):
            self.rules = []
        
        def add_transition_rule(self, rule):
            self.rules.append(rule)
            print(f"Model: Added {rule}")
    
    model = MockModel()
    
    # Add your specific transition with random parameters
    random_rule = create_random_transition_rule(
        from_state=PatientState.LINE,
        to_state=PatientState.TREATMENT_1,
        model=model,
        probability_config={
            'distribution': ('beta', 5, 2),  # High probability (Beta(5,2))
            'clip_range': (0.8, 1.0)  # Always at least 80%
        },
        duration_config={
            'distribution': ('normal', 7, 3),  # ~1 week with variation
            'clip_range': (0, 30)  # No more than 30 days
        }
    )

=== EXAMPLE 1: Original use case with random parameters ===
Original rule:
  TransitionRule(from_state=PatientState.LINE, to_state=PatientState.TREATMENT_1, min_duration=0, probability=1.0)

Random rule 1 (uniform probability):
  TransitionRule(from_state=PatientState.LINE, to_state=PatientState.TREATMENT_1, min_duration=0, probability=0.785)

Random rule 2 (beta probability, triangular duration):
  TransitionRule(from_state=PatientState.LINE, to_state=PatientState.TREATMENT_1, min_duration=9, probability=0.231)

Random rule 3 (normal distributions):
  TransitionRule(from_state=PatientState.LINE, to_state=PatientState.TREATMENT_1, min_duration=2, probability=0.935)


=== EXAMPLE 2: Full model transitions ===
line → treatment_1: prob=0.954, duration=16
treatment_1 → treatment_2: prob=0.247, duration=157
treatment_1 → progression: prob=0.355, duration=1592
treatment_2 → progression: prob=0.828, duration=112
progression → death: prob=0.930, duration=86


=== EXAMPLE 3: Batch generation fo

In [11]:
"""
Patient Treatment Flow Framework using Finite State Machines

This framework models patient journeys through treatment as FSMs, allowing 
easy tracking of states, transitions, and outcomes over time.
"""

from dataclasses import dataclass, field
from typing import Dict, List, Optional, Callable
from enum import Enum
import random


# ============================================================================
# STATE DEFINITIONS
# ============================================================================

class PatientState(Enum):
    """Define all possible patient states
    TODO: add aspects that determine treatment choice etc. and make it a random variable"""
    LINE = "diagnosed"
    TREATMENT_1 = "treatment_1"
    TREATMENT_2 = "treatment_2"
    TREATMENT_3 = "treatment_3"
    REMISSION = "remission"
    DECEASED = "deceased"


# ============================================================================
# PATIENT CLASS (Finite State Machine)
# ============================================================================

@dataclass
class Patient:
    """
    Represents a patient as a finite state machine.
    Tracks current state, history, and time spent in each state.
    """
    id: int
    current_state: PatientState = PatientState.LINE
    time_in_current_state: int = 0
    state_history: List[tuple] = field(default_factory=list)  # (state, duration)
    total_time: int = 0
    
    def transition_to(self, new_state: PatientState):
        """
        Move patient to a new state, recording history.
        """
        # Record the state we're leaving
        self.state_history.append((self.current_state, self.time_in_current_state))
        
        # Transition to new state
        self.current_state = new_state
        self.time_in_current_state = 0
    
    def advance_time(self, time_units: int = 1):
        """
        Progress time for this patient in their current state.
        """
        self.time_in_current_state += time_units
        self.total_time += time_units
    
    def is_terminal(self) -> bool:
        """
        Check if patient is in a terminal state (deceased or long-term remission).
        """
        return self.current_state in [PatientState.DECEASED, PatientState.REMISSION]


# ============================================================================
# TRANSITION RULES
# ============================================================================

@dataclass
class TransitionRule:
    """
    Defines when and how a patient moves from one state to another.
    """
    from_state: PatientState
    to_state: PatientState
    min_duration: int  # Minimum time before transition can occur
    probability: float  # Probability of transition per time unit after min_duration
    condition: Optional[Callable[[Patient], bool]] = None  # Optional custom condition
    
    def can_transition(self, patient: Patient) -> bool:
        """
        Check if this transition rule applies to the patient.
        """
        # Must be in the correct state
        if patient.current_state != self.from_state:
            return False
        
        # Must have spent minimum time in state
        if patient.time_in_current_state < self.min_duration:
            return False
        
        # Check custom condition if provided
        if self.condition and not self.condition(patient):
            return False
        
        # Apply probability
        return random.random() < self.probability
    

# ============================================================================
# FLOW MODEL
# ============================================================================

class TreatmentFlowModel:
    """
    Orchestrates patient treatment flows by defining transition rules
    and simulating patient journeys through states.
    """
    
    def __init__(self):
        self.patients: List[Patient] = []
        self.transition_rules: List[TransitionRule] = []
        self.current_time: int = 0
        
    def add_transition_rule(self, rule: TransitionRule):
        """Add a transition rule to the model"""
        self.transition_rules.append(rule)
    
    def add_patient(self, patient: Patient):
        """Add a patient to the simulation"""
        self.patients.append(patient)
    
    def step(self):
        """
        Advance simulation by one time unit.
        For each patient, check transitions and advance time.
        """
        self.current_time += 1
        
        for patient in self.patients:
            # Skip patients in terminal states
            if patient.is_terminal():
                continue
            
            # Check all transition rules to see if patient should change state
            transitioned = False
            for rule in self.transition_rules:
                if rule.can_transition(patient):
                    patient.transition_to(rule.to_state)
                    transitioned = True
                    break  # Only one transition per time step
            
            # Always advance time for the patient
            patient.advance_time(1)
    
    def run(self, max_time: int):
        """
        Run the simulation for a specified number of time units.
        """
        for _ in range(max_time):
            self.step()
    
    # ========================================================================
    # DATA READOUT METHODS
    # ========================================================================
    
    def get_state_distribution(self) -> Dict[PatientState, int]:
        """Get current count of patients in each state"""
        distribution = {state: 0 for state in PatientState}
        for patient in self.patients:
            distribution[patient.current_state] += 1
        return distribution
    
    def get_average_time_in_state(self, state: PatientState) -> float:
        """Calculate average time patients spend in a given state"""
        times = []
        for patient in self.patients:
            for hist_state, duration in patient.state_history:
                if hist_state == state:
                    times.append(duration)
        return sum(times) / len(times) if times else 0
    
    def get_patient_outcomes(self) -> Dict[str, int]:
        """Summarize final outcomes for all patients"""
        outcomes = {"deceased": 0, "remission": 0, "in_treatment": 0}
        for patient in self.patients:
            if patient.current_state == PatientState.DECEASED:
                outcomes["deceased"] += 1
            elif patient.current_state == PatientState.REMISSION:
                outcomes["remission"] += 1
            else:
                outcomes["in_treatment"] += 1
        return outcomes
    
    def get_patient_journeys(self) -> List[Dict]:
        """Get detailed journey information for each patient"""
        journeys = []
        for patient in self.patients:
            journey = {
                "patient_id": patient.id,
                "current_state": patient.current_state.value,
                "total_time": patient.total_time,
                "history": [(s.value, d) for s, d in patient.state_history]
            }
            journeys.append(journey)
        return journeys
    
    def print_summary(self):
        """Print a summary of the simulation results"""
        print(f"\n{'='*60}")
        print(f"SIMULATION SUMMARY (Time: {self.current_time})")
        print(f"{'='*60}")
        
        print(f"\nTotal Patients: {len(self.patients)}")
        
        print("\nCurrent State Distribution:")
        for state, count in self.get_state_distribution().items():
            print(f"  {state.value:20s}: {count:4d} patients")
        
        print("\nPatient Outcomes:")
        for outcome, count in self.get_patient_outcomes().items():
            print(f"  {outcome:20s}: {count:4d} patients")
        
        print("\nAverage Time in Each State:")
        for state in PatientState:
            avg_time = self.get_average_time_in_state(state)
            print(f"  {state.value:20s}: {avg_time:6.1f} time units")


# ============================================================================
# EXAMPLE USAGE
# ============================================================================

def create_example_model():
    """
    Create an example treatment flow model with realistic transitions.
    This demonstrates how to set up a simple treatment pathway.
    """
    model = TreatmentFlowModel()
    
    # Define transition rules for a typical treatment flow
    # Format: from_state -> to_state (min_duration, probability per time unit)

    # Simple usage matching your original structure

    generator = RandomTransitionRuleGenerator()
    
    model.add_transition_rule(generator.generate_rule(
        from_state=PatientState.LINE,
        to_state=PatientState.TREATMENT_1,
        prob_distribution=('uniform', 0.8, 1.0),  # 80-100% probability
        duration_distribution=('triangular', 0, 7, 14)  # 0-14 days, most likely 7
    ))
    
    # From diagnosis to first treatment (immediate, certain)
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.LINE,
        to_state=PatientState.TREATMENT_1,
        min_duration=0,
        probability=1.0
    ))
    
    # Treatment 1 outcomes (after minimum 3 time units)
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_1,
        to_state=PatientState.TREATMENT_2,
        min_duration=3,
        probability=0.15  # 15% chance per time unit to progress
    ))
    
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_1,
        to_state=PatientState.DECEASED,
        min_duration=3,
        probability=0.02  # 2% mortality risk per time unit
    ))
    
    # Treatment 2 outcomes
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_2,
        to_state=PatientState.TREATMENT_3,
        min_duration=4,
        probability=0.10
    ))
    
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_2,
        to_state=PatientState.REMISSION,
        min_duration=4,
        probability=0.08  # 8% chance of remission per time unit
    ))
    
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_2,
        to_state=PatientState.DECEASED,
        min_duration=4,
        probability=0.03
    ))
    
    # Treatment 3 outcomes
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_3,
        to_state=PatientState.REMISSION,
        min_duration=5,
        probability=0.12
    ))
    
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_3,
        to_state=PatientState.DECEASED,
        min_duration=5,
        probability=0.05
    ))
    
    return model


if __name__ == "__main__":
    # Create the model
    model = create_example_model()
    
    # Add 100 patients to the simulation
    for i in range(100):
        model.add_patient(Patient(id=i))
    
    # Run simulation for 50 time units
    print("Running simulation...")
    model.run(max_time=50)
    
    # Display results
    model.print_summary()
    
    # Example: Get detailed journey for first patient
    print("\n" + "="*60)
    print("EXAMPLE: First Patient's Journey")
    print("="*60)
    journeys = model.get_patient_journeys()
    first_journey = journeys[0]
    print(f"Patient {first_journey['patient_id']}:")
    print(f"  Current State: {first_journey['current_state']}")
    print(f"  Total Time: {first_journey['total_time']}")
    print(f"  Journey:")
    for state, duration in first_journey['history']:
        print(f"    {state:20s} for {duration} time units")

Running simulation...

SIMULATION SUMMARY (Time: 50)

Total Patients: 100

Current State Distribution:
  diagnosed           :    0 patients
  treatment_1         :    0 patients
  treatment_2         :    0 patients
  treatment_3         :    1 patients
  remission           :   60 patients
  deceased            :   39 patients

Patient Outcomes:
  deceased            :   39 patients
  remission           :   60 patients
  in_treatment        :    1 patients

Average Time in Each State:
  diagnosed           :    0.0 time units
  treatment_1         :    8.7 time units
  treatment_2         :    7.9 time units
  treatment_3         :    9.8 time units
  remission           :    0.0 time units
  deceased            :    0.0 time units

EXAMPLE: First Patient's Journey
Patient 0:
  Current State: remission
  Total Time: 26
  Journey:
    diagnosed            for 0 time units
    treatment_1          for 7 time units
    treatment_2          for 8 time units
    treatment_3          for 

In [11]:
import math

def fibonacci(n):
    if n <= 1:
        return n
    prev, curr = 0, 1
    for _ in range(2, n + 1):
        prev, curr = curr, prev + curr
    return curr

[(fibonacci(n), 2*fibonacci(n)//(1+math.sqrt(5))) for n in range(11)]

[(0, 0.0),
 (1, 0.0),
 (1, 0.0),
 (2, 1.0),
 (3, 1.0),
 (5, 3.0),
 (8, 4.0),
 (13, 8.0),
 (21, 12.0),
 (34, 21.0),
 (55, 33.0)]